In [ ]:
import json
import boto3
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.cache_handler import CacheHandler

# Custom null cache handler
class NullCacheHandler(CacheHandler):
    def get_cached_token(self):
        return None

    def save_token_to_cache(self, token_info):
        pass

# Initialize S3 client
s3 = boto3.client('s3')

def lambda_handler(event, context):
    # Spotify API credentials
    CLIENT_ID = 'ceae9ed1f9ca47589364052977c3e111'
    CLIENT_SECRET = 'fba2d5d3c8e540b28566c23cdf375bfb'

    # Authenticate with Spotify API using a null cache handler
    auth_manager = SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        cache_handler=NullCacheHandler()  # Suppress caching
    )
    sp = Spotify(auth_manager=auth_manager)

    try:
        # Extract artist_name from the event
        artist_name = 'Arijit Singh'
        if not artist_name:
            return {
                'statusCode': 400,
                'body': "Error: 'artist_name' is required in the event payload."
            }

        # Search for the artist
        result = sp.search(q=artist_name, type='artist', limit=1)
        artists = result.get('artists', {}).get('items', [])
        if not artists:
            return {
                'statusCode': 404,
                'body': f"Error: No artist found with the name '{artist_name}'."
            }

        artist_id = artists[0]['id']

        # Fetch the artist's top tracks
        top_tracks = sp.artist_top_tracks(artist_id, country='IN')

        # Convert to JSON
        top_tracks_json = json.dumps(top_tracks)

        # Upload to S3
        bucket_name = 'spotify-extractdata'
        file_name = f'spotify_artist_{artist_name.replace(" ", "_").lower()}_top_tracks.json'
        s3.put_object(Bucket=bucket_name, Key=file_name, Body=top_tracks_json)

        return {
            'statusCode': 200,
            'body': f"Top tracks for '{artist_name}' uploaded to S3: {bucket_name}/{file_name}"
        }

    except Exception as e:
        return {
            'statusCode': 500,
            'body': f"Error: {str(e)}"
        }